In [2]:
import requests
import json
import urllib
#replace client_id with Client Key given at https://myanimelist.net/apiconfig
client_id = "0bd76a18d2f4d2bb27ca27105be29188"

base_url = "https://api.myanimelist.net/v2/"
auth_header = {"X-MAL-CLIENT-ID" : client_id}

In [2]:
# Get Anime List of turtlemage

#user = "turtlemage"
#content_type = "animelist"
# Construct url to HTTP GET
#url = base_url + "users/" + user + "/" + content_type

#fields = {
#    "fields" : "list_status"
#}
# Send HTTP GET Request
#r = requests.get(url, headers = auth_header, params = fields)
#print(r.status_code)
#
#full_json = r.json()
#next_request_url = full_json['paging']['next']
#print(full_json)




# Loop for paging through queries that may require more than 1 Request
#while 'paging' in full_json:
#    paging = full_json['paging']
#    new_url = paging['next']
#    r = requests.get(new_url, headers = auth_header)
# Status Code 200 means that the request is Successful (i.e not Rate Limited)
#    if r.status_code == 200:
# This appends the previous json with the newly obtained json
#        full_json['data'] += r.json()['data']
#        if 'next' in r.json()['paging']:
#            full_json['paging'] = r.json()['paging']
#        else:
#            full_json.pop('paging')
#print(full_json)


In [33]:
def get_user_list_ids(client_id, user, content_type='anime', fields='list_status', returnJSON=True, return_type='id'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    user         - username of user
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    fields       - list of fields to include in returned data, string deliminated by commas
    returnJSON   - boolean deciding whether to return raw JSON or to return simplified data
    return_type  - recommended id or title, determines the keys of formatted dictionary        
    """ 
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'animelist'
    else:
        content = 'mangalist'
    data_request = {'fields' : fields}
    # Create request url, to pass HTTP GET request with header and fields
    url = base_url + "users/" + user + "/" + content
    # Initial HTTP request
    r = requests.get(url, headers = auth_header, params = data_request)
    if r.status_code != 200:
        raise Exception("HTTP GET Error : "+str(r.status_code))
    full_json = r.json()
    # Loops through all pages for lists that are too large for one HTTP GET
    while 'paging' in full_json:
        paging = full_json['paging']
        new_url = paging['next']
        r = requests.get(new_url, headers = auth_header)
        if r.status_code != 200:
            raise Exception("HTTP GET Error : "+str(r.status_code))
        full_json['data'] += r.json()['data']
        if 'next' in r.json()['paging']:
            full_json['paging'] = r.json()['paging']
        else:
            full_json.pop('paging')
    # Returns raw data if left alone
    if returnJSON:
        return full_json
    # Attempt at Cleaning the json up a bit for usability - not necessarily too useful
    else:
        shows = full_json['data']
    field_list = fields.split(',')
    show_list = {}
    for node in full_json['data']:
        try:
            show = node['node'][return_type]
            show_list[show] = {}
        except:
            raise Exception('return_type not valid - must be id, title, or other field type')
        for field in field_list:
            show_list[show][field] = node['node'][field]
    return show_list





def get_show_id(client_id, show_title, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_title   - title of show that an ID needs to be found for
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """ 
    # Allowing lists to be passed thru to simplify future code
    if type(show_title) is list:
        show_ids = []
        for show in show_title:
            show_id.append(get_show_id(client_id, show, content_type))
        return show_ids
    # Creating HTTP GET header
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Create request url, to pass HTTP GET request with header and fields
    # Uses MAL's search feature to find show
    url = base_url + content + "?q=" +  urllib.parse.quote(show_title.lower().split()[0])
    # Initial HTTP request
    r = requests.get(url,headers = auth_header)
    if r.status_code != 200:
        raise Exception("HTTP GET Error : "+str(r.status_code))
    for node in r.json()['data']:
        if node['node']['title'] == show_title:
            return node['node']['id']
    raise Exception("Title not Found")
    
    
    
    
    
    
def get_show_stats(client_id, show_ids=[], show_title=[], content_type='anime', stat='all'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_id      - integer relating to the index in MAL's database
    show_title   - title of show that an ID needs to be found for
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    stat         - statistic to be found (all returns default stats)
    """ 
    if type(show_ids) == int:
        show_ids = [show_ids]
    if show_ids == [] and show_title == []:
        raise Exception("Must submit an id or title")
    elif show_ids == []:
        for show in show_title:
            show_ids.append(get_show_id(client_id, show_title, content_type))
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'anime/'
    else:
        content = 'manga/'
    # Loop through all shows and request (cant seen to figure out how to batch them other than using a temp holder acct)
    results = []
    for show in show_ids:
        # Create request url, to pass HTTP GET request with header and fields
        url = base_url + content + str(show)
        # Initial HTTP request
        r = requests.get(url, headers = auth_header, params = {'fields' : stat})
        results.append(r.json())
    if stat == 'all':
        return results
    else:
        amended_results = []
        for show in results:
            amended_results.append(show[stat])
        return amended_results
    
def get_genre(client_id, show_ids, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_ids     - list of show indexes to get genre info about
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """ 
    # If only checking 1 show, puts single show into list wrapper
    if type(show_ids) == int:
        show_ids = [show_ids]
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    genre_dict = {}
    # For each show in list, HTTP GET genre
    for show in show_ids:
        url = base_url + content + "/" + str(show) + "?fields=genres"
        r = requests.get(url, headers=auth_header)
        genre_dict[show] = r.json()['genres']
    return genre_dict




def get_titles(client_id, show_id, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_id      - id of shows that alternate titles are needed for
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """ 
    # If only checking 1 show, puts single show into list wrapper
    if type(show_ids) == int:
        show_ids = [show_ids]
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    title_dict = {}
    # HTTP GET alternative titles for each show entry
    for show in show_id:
        url = base_url + content + "/" + str(show_id) + "?fields=title,alternative_titles"
        r = requests.get(url, headers=auth_header)
        title_dict[show] = r.json()['alternative_titles']
        title_dict[show]['popular_title'] = r.json()['title']
    return title_dict

def get_genre_preferences(client_id, user, content_type='anime', only_watched=True):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    user         - MAL username that is being analysed
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    only_watched - Set False if all entries (including Dropped and Plan to Watch) are to be analysed
    """
    # Get user animelist
    show_list = {}
    user_animelist = get_user_list_ids(client_id, user, content_type, fields='list_status')
    if only_watched:
        for node in user_animelist['data']:
            if node['list_status']['status'] == "completed":
                show_list[node['node']['id']] = node['list_status']['score']
    else:
        for node in user_anilist['data']:
            show_list[node['node']['id']] = node['list_status']['score']
    # Get show ids
    show_ids = []
    for key in show_list.keys():
        show_ids.append(key)
    # get genre info
    genre_list = get_genre(client_id, show_ids, content_type)
    genres = {}
    # iterate through genres to get count of number of shows with each genre tag and average user score
    for show in genre_list.keys():
        for genre in genre_list[show]:
            if genre['id'] not in genres.keys():
                genres[genre['id']] = {'genre': genre, 'count' : 1, 'avg_score' : show_list[show]}
            else:
                count = genres[genre['id']]['count'] + 1
                avg_score = (genres[genre['id']]['avg_score'] * (count - 1) + show_list[show]) / count
                genres[genre['id']]['count'] = count
                genres[genre['id']]['avg_score'] = avg_score
    return genres      

def get_vintage_preferences(client_id, user, content_type='anime', only_watched=True):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    user         - MAL username that is being analysed
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    only_watched - Set False if all entries (including Dropped and Plan to Watch) are to be analysed
    """
    # get user list and season info
    show_list = {}
    user_animelist = get_user_list_ids(client_id, user, content_type, fields='list_status,start_season')
    season_list = {}
    if only_watched:
        for node in user_animelist['data']:
            if node['list_status']['status'] == "completed":
                show_list[node['node']['id']] = node['list_status']['score']
                season_list[node['node']['id']] = node['node']['start_season']
    else:
        for node in user_anilist['data']:
            show_list[node['node']['id']] = node['list_status']['score']
            season_list[node['node']['id']] = node['node']['start_season']
    show_ids = []
    seasons = {}
    # get show ids
    for key in show_list.keys():
        show_ids.append(key)
    # for each season, count number of shows and average rating
    for show in season_list.keys():
        season = str(season_list[show]['year'])+" "+season_list[show]['season']
        if season not in seasons.keys():
            seasons[season] = {'season': season_list[show], 'count' : 1, 'avg_score' : show_list[show]}
        else:
            count = seasons[season]['count'] + 1
            avg_score = (seasons[season]['avg_score'] * (count - 1) + show_list[show]) / count
            seasons[season]['count'] = count
            seasons[season]['avg_score'] = avg_score
    return seasons     
def get_related_shows(client_id, show_ids, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_ids     - List of show ids to check
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """
    # If only checking 1 show, puts single show into list wrapper
    if type(show_ids) == int:
        show_ids = [show_ids]
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    franchises = {}
    # HTTP GET each show's related series
    for show in show_ids:
        url = base_url + content + "/" + str(show) + "?fields=related_" + content
        r = requests.get(url, headers=auth_header)
        if show not in franchises:
            franchises[show] = r.json()['related_'+content]
    for show in franchises.keys():
        rel_list = []
        for node in franchises[show]:
            rel_list.append(node['node']['id'])
        franchises[show] = rel_list
    return franchises
def get_show_pictures(client_id, show_ids, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_ids     - List of show ids to check
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """
    # If only checking 1 show, puts single show into list wrapper
    if type(show_ids) == int:
        show_ids = [show_ids]
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    images = {}
    # HTTP GET each show's related series
    for show in show_ids:
        url = base_url + content + "/" + str(show)
        r = requests.get(url, headers=auth_header)
        images[show] = r.json()['main_picture']
    return images
def get_show_studio(client_id, show_ids, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_ids     - List of show ids to check
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """
    # If only checking 1 show, puts single show into list wrapper
    if type(show_ids) == int:
        show_ids = [show_ids]
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    studios = {}
    # HTTP GET each show's Production Studios
    for show in show_ids:
        url = base_url + content + "/" + str(show)
        r = requests.get(url, headers=auth_header, params={'fields':'studios'})
        studios[show] = r.json()['studios']
    return studios

def get_preference(client_id, user, field, content_type='anime', only_watched=True):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    user         - MAL username that is being analysed
    field        - string for field that is wanted
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    only_watched - Set False if all entries (including Dropped and Plan to Watch) are to be analysed
    """
    # get user list and season info
    show_list = {}
    user_animelist = get_user_list_ids(client_id, user, content_type, fields='list_status,'+field)
    field_list = {}
    if only_watched:
        for node in user_animelist['data']:
            if node['list_status']['status'] == "completed":
                show_list[node['node']['id']] = node['list_status']['score']
                field_list[node['node']['id']] = node['node'][field]
    else:
        for node in user_anilist['data']:
            show_list[node['node']['id']] = node['list_status']['score']
            field_list[node['node']['id']] = node['node'][field]
    show_ids = []
    fields = {}
    # get show ids
    for key in show_list.keys():
        show_ids.append(key)
    # iterate through genres to get count of number of shows with each genre tag and average user score
    for show in field_list.keys():
        for field_ in field_list[show]:
            if field_['id'] not in fields.keys():
                fields[field_['id']] = {field : field_, 'count' : 1, 'avg_score' : show_list[show]}
            else:
                count = fields[field_['id']]['count'] + 1
                avg_score = (fields[field_['id']]['avg_score'] * (count - 1) + show_list[show]) / count
                fields[field_['id']]['count'] = count
                fields[field_['id']]['avg_score'] = avg_score
    return_field = []
    for field in fields.keys():
        return_field.append(fields[field])
    return return_field       

def get_top_shows(client_id, numShows, content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    numShows     - number of shows (from the top) to return
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """ 
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    # Until numShows is reached, HTTP GET more
    if numShows < 500:
        iteration = numShows
    else:
        iteration = 500
    url = base_url + content + '/ranking?ranking_type=all&limit=' + str(iteration)
    r = requests.get(url, headers=auth_header)
    show_list = []
    i = 0
    while i < numShows:
        for node in r.json()['data']:
            if i < numShows:
                show_list.append(node['node']['id'])
                i = i + 1
        if i < numShows:
            r = requests.get(r.json()['paging']['next'], headers=auth_header)
    return show_list

def get_top_shows_stat(client_id, show_num, stat='', content_type='anime'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    show_num     - number of shows (from the top) to return
    stat         - statistic to get
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    """ 
    # Define Content Type for url definition
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Creating Authentication header for HTTP GET
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    # Until numShows is reached, HTTP GET more
    if show_num < 500:
        iteration = show_num
    else:
        iteration = 500
    url = base_url + content + '/ranking?ranking_type=all&limit=' + str(iteration) + "fields=" + stat
    r = requests.get(url, headers=auth_header)
    show_list = []
    i = 0
    while i < show_num:
        for node in r.json()['data']:
            if i < show_num:
                show_list.append(node['node'])
                i = i + 1
        if i < show_num:
            r = requests.get(r.json()['paging']['next'], headers=auth_header)
    return show_list